## ACF model

Firstly we define the functions that appear in our pseudocode

In [1]:
import numpy as np
from itertools import compress




class Sector1:
    """
    Stores the parameters of Sector 1
    """

    def __init__(self, alpha_1 = 0.5,     # Price/performance sensitivity of demand
                 eta = 1.5,               # Common parameter in pricing routine
                 c = 10,                # Unit production cost
                 phi = 0.5,               # Relative importance imitation vs inner R&D
                 epsilon = 0.75,          # Entry cost for new imitative entrants
                 landa = 0.05):           # Probability of entering doing innovation


        self.alpha, self.eta, self.c, self.phi = alpha_1, eta, c, phi
        self.epsilon, self.landa = epsilon, landa

        self.firms = list()  # Creates an empty list of firms
        self.m_prices = list()
        self.herfin = list()
        self.m_RD = list()
        self.m_r = list()
    
    def shares(self):
        
        return np.array([firm.s for firm in self.firms])
    
    def x_max(self):                      # Returns max technological level
        
        return max([firm.x for firm in self.firms])
    
    def x_min(self):                      #Returns min technology
        
        return min([firm.x for firm in self.firms])

    def R_max(self):                      # Returns max R_D investment
        
        return max([firm.R_D for firm in self.firms])

    def mean_knowledge(self):            # Retturns average firm knowledge
        
        knowledge = np.array([firm.gamma for firm in self.firms])
        xs = np.array([firm.x for firm in self.firms])

        return np.sum(xs * knowledge)
    
    def x_max_prev(self):            #Returns max x of t-1
        qualities = list()
        for firm in self.firms:
            if len(firm.xs) != 0:
                qualities.append(firm.xs[-1])
                
        return max(qualities)
    
    def medium_prices(self):
    
        prices = np.array([firm.p for firm in self.firms])
        self.m_prices.append(np.sum(self.shares() * prices))
    
    def medium_RD(self):
        RDs = np.array([firm.R_D for firm in self.firms])
        self.m_RD.append(np.sum(self.shares() * RDs))
        
    def medium_r(self):
        rs = np.array([firm.r for firm in self.firms])
        self.m_r.append(np.sum(self.shares() * rs))
        
    def cal_herfin(self):
        
        herfin_index = np.sum((self.shares() * 100)**2)
        self.herfin.append(herfin_index)
        

class Sector2:
    """
    Stores the parameters of Sector 2
    """

    def __init__(self, alpha_2=0.5,      # Price/performance sensitivity of demand
                 landa=0.05,             # Probability of entry doing innovation
                 delta= 1.5,               # Common parameter in pricing routine
                 a=3,                    # parameter beta-distribution
                 b=1):                   # parameter beta-distribution
        

        self.alpha, self.landa, self.delta = alpha_2, landa, delta,
        self.a, self.b = a, b

        self.dumb = 0
        self.firms = list()             # Creates an empty list firms
        self.m_prices = list()
        self.herfin = list()
        
    def shares(self):                   #Returns shares of the sector
        
        return np.array([firm.s for firm in self.firms])
    
    def y_max(self):                    # Returns max amchine quality in sector
        
        return max([firm.y for firm in self.firms])

    def p_max(self):                    # Returns max price in sector
        
        return max([firm.p for firm in self.firms])

    def mean_fit(self):                 # Returns average firm fitness
        
        fits = np.array([firm.f for firm in self.firms])
        shares = np.array([firm.s for firm in self.firms])

        return np.sum(fits * shares)
    
    def medium_prices(self):
    
        prices = np.array([firm.p for firm in self.firms])
        self.m_prices.append(np.sum(self.shares() *prices))
        
    def cal_herfin(self):
        
        herfin_index = np.sum((self.shares() * 100) ** 2)
        self.herfin.append(herfin_index)

class firm_S1:
    """
    Generates a capital-good firm
    """

    def __init__(self, Sector1,         # Defines the industry of the firm
                 pioneer=False,         # If == True is the first firm to enter the market
                 innovation=False,      # If == False firm imitates traits of other firm
                 share = 0):     
        
        # Lists to store variables' time series
        self.prices = list()
        self.quantities = list()
        self.profits = list()
        self.shares = list()
        self.xs = list()
        self.periods = 0
        
        #inititate variables
        self.profit = 0
        self.s_e = share
        self.s = 0
        self.q = 0
        self.c = 0

        if pioneer == True or innovation == True:  # Random selection of traits

            self.r = np.random.uniform(0, 1)
            self.sigma = np.random.uniform(0, 1)
            self.c_e = Sector1.c
            self.R_D = 0

            if pioneer: # Random x when firm IS pioneer
              
                self.x = np.random.uniform(0, 1)

            else:                                # Random x when firm is NOT pioneer
                self.x = np.random.uniform(0, Sector1.x_max_prev())


        else:                                    # imitation of traits

            probabilities = np.array([firm.s for firm in Sector1.firms])
            firm_imitate = np.random.choice(Sector1.firms, p = probabilities)

            self.r, self.sigma, self.x = firm_imitate.r, firm_imitate.sigma, firm_imitate.xs[-1]

            self.q_e = firm_imitate.quantities[-1]

            self.R_D = firm_imitate.R_D

            self.c_e = Sector1.c + Sector1.epsilon* (self.R_D / self.q_e)

    def R_D_investment(self):                   # Calculates R_D investment

        self.R_D = self.r * self.profits[-1]

    def expected_c(self, Sector1):             # Calculates expcted cost

        self.q_e = self.quantities[-1]
        self.c_e = Sector1.c + (self.R_D / self.q_e)

    def expected_rivals(self, Sector1, creating = False):   # Creates the set of perceived rivals

        rivals = list()
        
        if creating == True:                   
            for firm in Sector1.firms:
                if firm == self:
                    continue
                else:
                    if abs(firm.x - self.x) <= self.sigma * Sector1.x_max():
                        rivals.append(firm)
                    else:
                        continue
        else:
            for firm in Sector1.firms[:-1]:
                if firm == self:
                    continue
                else:
                    if abs(firm.xs[-1] - self.x) <= self.sigma * Sector1.x_max_prev():
                        rivals.append(firm)
                    else:
                        continue

        self.rivals = rivals

    def mark_up(self, Sector1):              # Calculates Cournot mark-up
        
        for firm in Sector1.firms:
            
            if len(firm.shares) != 0:
                firm.s_e = firm.shares[-1]
        
        rivals_shares = sum(firm.s for firm in self.rivals) 
        self.mu = (Sector1.eta + rivals_shares) / ((Sector1.eta + rivals_shares) - self.s_e)

    def price(self):                        # Calculates price

        self.p = self.mu * self.c_e

    def knowledge(self, Sector1):           # Generates new knowledge

        imitation = (Sector1.x_max() - self.x) / self.x
        
        if self.R_D == 0:
            research = 0
        else:
            research = self.R_D / Sector1.R_max()
        
        innovation = ((Sector1.phi * imitation) + ((1 - Sector1.phi) * research))
        if innovation == 0:
            self.gamma = 0
            
        else:
            slope = 1 / innovation
            new_knowledge = 1 - np.random.power(slope)

            self.gamma = new_knowledge


class firm_S2:
    """
    Generates a consumption-good firm
    """

    def __init__(self, Sector1,                # Defines capital-good industry
                 Sector2,                      # Defines consumption-good industry
                 pioneer=False,                # If it is the first to enter in the market
                 innovation=False,             # If == False firm imitates traits of other firm
                 share=0):                     # Initial market share
        
        # Lists to store variables' time series
        self.periods = 0
        self.shares = list()
        self.prices = list()
        self.Xs = list()
        

        self.s = share

        if pioneer == True or innovation == True:  # Random traits

            self.ro = np.random.beta(Sector2.a, Sector2.b)
                                                     
            self.X = np.random.uniform(Sector1.x_min(), Sector1.x_max())


        else:  # Imitate traits
            
            p_shares = np.array([firm.s for firm in Sector2.firms])

            firm_imitate = np.random.choice(Sector2.firms, p = p_shares)
            
            self.ro, self.X = firm_imitate.ro, firm_imitate.X

    def understandable(self, Sector1, Sector2):            # Selection of undertandable machines

        S1_understand = list()

        for firm in Sector1.firms:

            if abs(self.X - firm.x) <= self.ro * Sector1.x_max():
                S1_understand.append(firm)
            else:
                pass
            
        self.understand = S1_understand
        
        if len(S1_understand) == 0:
            Sector2.dumb += 1
            

    def buy_machine(self, Sector1, Sector2):    # Buys a machine from Sector 1 


            xs = np.array([firm.x for firm in self.understand], dtype=float)
            prices = np.array([firm.p for firm in self.understand], dtype=float)
            price_param = prices / np.sum(prices)
            
            probabilities = Sector1.alpha * xs   + (1 - Sector1.alpha) * (1 - price_param)

            probabilities /= np.sum(probabilities)

            buy = np.random.choice(self.understand, p=probabilities)

            self.c, self.X, self.y = buy.p, buy.x, buy.x

            buy.q += 1

    def price(self, Sector2):                   # Pricing routine

        self.p = (Sector2.delta / (Sector2.delta - self.s)) * self.c

    def fitness(self, Sector2):                 # Firm fitness
        
        self.f = Sector2.alpha * (self.y / Sector2.y_max()) + (1 - Sector2.alpha) * (1 - (self.p / Sector2.p_max()))
                 

def understanding_set(Sector1, Sector2):
    """
    Function for determining the understandable set of machines of Sector2
    """
        
    compresor = list()
    
    for firm in Sector2.firms:
        firm.understandable(Sector1, Sector2)
        if len(firm.understand) == 0:
            compresor.append(0)
        else:
            compresor.append(1)
            
    Sector2.firms = list(compress(Sector2.firms, compresor))
    
    recalculate_shares(Sector2)

def recalculate_xs(Sector1):
    """
    Recalculates x of firm_S1 to make it sum up to one
    
    """
    total = np.sum(np.array([firm.x for firm in Sector1.firms]))
    for firm in Sector1.firms:
        firm.x /= total
    
    
def recalculate_Xs(Sector2):
    """
    Recalculates Xs of Sector 2 so as to they sum up to one
    
    """
    total = np.sum(np.array([firm.X for firm in Sector2.firms]))
    for firm in Sector2.firms:
        firm.X /= total
    
    
    
def recalculate_shares(Sector):
    """
    Recalculate shares of the sector so as to they sum upt to one
    """
    
    total_shares = sum([firm.s for firm in Sector.firms])
    for firm in Sector.firms:
        firm.s /= total_shares
    
def create_market(n_1, n_2, Sector1, Sector2):
    """
    Function for creating the markets
    """
    
    for i in range(n_1):
        Sector1.firms.append(firm_S1(Sector1, pioneer = True, share = 1/n_1))
    
    recalculate_xs(Sector1)
    for firm in Sector1.firms:
        firm.expected_rivals(Sector1, creating = True)
        firm.mark_up(Sector1)
        firm.price()
        firm.knowledge(Sector1)
        
    for firm in range(n_2):
        Sector2.firms.append(firm_S2(Sector1, Sector2, pioneer = True, share= 1/n_2))
    recalculate_Xs(Sector2)
    
    understanding_set(Sector1, Sector2)

    
    for firm in Sector2.firms:
        firm.buy_machine(Sector1, Sector2)
    for firm in Sector2.firms:
        firm.price(Sector2)
    for firm in Sector2.firms:
        firm.fitness(Sector2)
    
def operate_S1(Sector1):
    """
    Operates firm of Sector 1 each period
    """
    
    if len(Sector1.firms[:-1]) != 0:

        for firm in Sector1.firms[:-1]:   # For all firms except the new entrant 
            firm.R_D_investment()
            firm.expected_c(Sector1)

    for firm in Sector1.firms:           
        
        firm.expected_rivals(Sector1)
        firm.mark_up(Sector1)
        firm.price()
        firm.knowledge(Sector1)


def operate_S2(Sector1, Sector2):
    """
    Operates firm of Sector 2 each period
    """
    understanding_set(Sector1, Sector2)
    
    for firm in Sector2.firms:
        firm.buy_machine(Sector1, Sector2)
        firm.price(Sector2)

    for firm in Sector2.firms:
        firm.fitness(Sector2)


    
def replicator_S1(Sector1):
    """
    Replicator dynamics of Sector 1
    """
    
    for firm in Sector1.firms:
        firm.x = (firm.gamma - Sector1.mean_knowledge()) * firm.x + firm.x


def replicator_S2(Sector2):
    """
    Replicator dynamics of Sector 2
    """
    
    for firm in Sector2.firms:
        firm.s = (firm.f - Sector2.mean_fit()) * firm.s + firm.s
        
    total_s = sum([firm.s for firm in Sector2.firms])  #Recalculate shares
    recalculate_shares(Sector2) 
            
    
def entry_Sector1(Sector1):
    """
    Each t initializes one capital good firm
    """
  
    innovation = np.random.binomial(1, Sector1.landa)
    if innovation == True or len(Sector1.firms) == 0:
        Sector1.firms.append(firm_S1(Sector1, innovation=True, share = 1/(len(Sector1.firms) + 1 )))
    else:
        Sector1.firms.append(firm_S1(Sector1, innovation=False, share = 1/(len(Sector1.firms) + 1)))

    recalculate_xs(Sector1)
    
def entry_Sector2(Sector1, Sector2):
    """
    Each t initializes one consumption good firm 
    """


    innovation = np.random.binomial(1, Sector2.landa)
        

    if innovation == True or len(Sector2.firms) == 0:
        Sector2.firms.append(firm_S2(Sector1, Sector2, innovation=True, share= 0.005))

    if innovation == False:
        Sector2.firms.append(firm_S2(Sector1, Sector2, innovation=False, share= 0.005))

    recalculate_shares(Sector2)
    recalculate_Xs(Sector2)
        
def exit_S1(Sector1):
    """
    Firms whose profit <= 0 exits the market
    """
    total_q = np.sum([x.q for x in Sector1.firms])     #Calculate market shares 
    for firm in Sector1.firms:
        firm.s = firm.q / total_q
        
    for firm in Sector1.firms:
        if firm.q == 0:
            firm.profit = 0
        else:
            firm.c = Sector1.c + (firm.R_D / firm.q)
            firm.profit = firm.q * (firm.p - firm.c)
    
    selector = list()

    for firm in Sector1.firms:
        if firm.profit <= 0:
            selector.append(0)
        else:
            selector.append(1)

    Sector1.firms = list(compress(Sector1.firms, selector))

    
    recalculate_shares(Sector1)
    recalculate_xs(Sector1)


def exit_S2(Sector2):
    """
    Firms whose market share <= 0.005 exits the market
    """
    selector = list()
    for firm in Sector2.firms:
        if firm.s <= 0.005:
            selector.append(0)
        else:
            selector.append(1)

    Sector2.firms = list(compress(Sector2.firms, selector))
    recalculate_shares(Sector2)
    recalculate_Xs(Sector2)

def save_previous(Sector1, Sector2):
    """
    Append values to time series before being changed by replicator dynamics
    """
    
    for firm in Sector1.firms:
        firm.xs.append(firm.x)
    for firm in Sector2.firms:
        firm.shares.append(firm.s)
        
def save_values(Sector1, Sector2):
    """
    Append  values to time series
    """
    Sector1.medium_prices()
    Sector2.medium_prices()
    Sector1.cal_herfin()
    Sector2.cal_herfin()
    Sector1.medium_RD()
    Sector1.medium_r()
    for firm in Sector1.firms:
        firm.prices.append(firm.p)
        firm.quantities.append(firm.q)
        firm.profits.append(firm.profit)
        firm.shares.append(firm.s)
        firm.periods += 1
        firm.q = 0

    for firm in Sector2.firms:
        firm.prices.append(firm.p)
        firm.periods += 1
        


Now we create a function to simulate the model

In [2]:
def simulate(n_1, n_2, T, innovation = "normal", research = "normal"):
    
    value_landa = {'normal': 0.05, 'high': 0.1, 'low': 0.01}
    value_phi = {'normal': 0.5, 'high': 0.9, 'low': 0.1}
    
    S1 = Sector1(landa = value_landa[innovation],phi = value_phi[research])
    S2 = Sector2(landa = value_landa[innovation])
        
    create_market(n_1, n_2, S1, S2)
    save_previous(S1, S2)
    replicator_S1(S1)
    replicator_S2(S2)
    exit_S1(S1)
    exit_S2(S2)
    save_values(S1, S2)

    for t in range(T):
        entry_Sector1(S1)
        entry_Sector2(S1, S2)
        operate_S1(S1)

        operate_S2(S1, S2)
        save_previous(S1, S2)
        replicator_S1(S1)
        replicator_S2(S2)
        exit_S1(S1)
        exit_S2(S2)
        save_values(S1, S2)

    return S1, S2

After defining all this functions we could run the model just with one line of code, as follows:

In [3]:
a, b = simulate(1,1,100)
